## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('..')
import stober_synthesis_utils as stober

In [3]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = '2024_10_10_random3try2.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [5]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

In [6]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [7]:
sample_table = pd.read_csv('SampleTable_2024_11_18_OptimizationRound4.csv')

In [8]:
stober.count_stock_vials(sample_table, 15000)

{'teos_count': 1, 'ammonia_count': 1, 'water_count': 1, 'ethanol_count': 2}

In [9]:
sample_table['ethanol_volume'].sum()/10000

2.754448927892372

## Load Tools

In [10]:
syringe_10 = syringe.HTTPSyringe.from_config(1, "../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(2, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_syringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_syringe.json")

In [11]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)

In [12]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3

## Load water syringe with water

In [31]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [113]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [112]:
water_syringe.set_pulsewidth(water_syringe.full_position+1, s = 10)

In [34]:
water_syringe.set_pulsewidth(water_syringe.full_position+150, s = 500)

In [35]:
water_syringe.load_syringe(750, water_syringe.full_position+150)

Loaded syringe, remaining volume 750 uL


## Load ammonia syringe

In [36]:
ammonia_syringe.set_pulsewidth(1420, s = 10)

In [37]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [111]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [92]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 10)

In [40]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+350, s = 500)

In [41]:
ammonia_syringe.load_syringe(370, ammonia_syringe.full_position+350)

Loaded syringe, remaining volume 370 uL


## Load TEOS syringe

In [94]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [110]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [96]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+1, s = 100)

In [97]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+200, s = 200)

In [46]:
teos_syringe.load_syringe(690, teos_syringe.full_position+200)

Loaded syringe, remaining volume 690 uL


## load ethanol syringe

In [47]:
#jubilee.park_tool()

In [48]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [114]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


In [50]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+1, s = 10)

In [51]:
mix_syringe.empty_position

1830

In [52]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+400, s = 500)

In [53]:
mix_syringe.load_syringe(3000, mix_syringe.full_position+400)

Loaded syringe, remaining volume 3000 uL


## Load labware


In [18]:
samples = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 2)
samples.manual_offset([(14.8, 174.0), (127.1, 174.4), (127.5, 104.7)])

-0.004650345326966636
New manual offset applied to septavialrev1_44_holder_2000ul


In [19]:
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks.manual_offset([(27.5, 263.5), (113.5, 263.5), (113.5, 208.0)])

-0.0
New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [20]:
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [21]:
teos_stocks = [stocks[0]]
ammonia_stocks = [stocks[1]]
water_stocks = [stocks[2]]
ethanol_stocks = [stocks[3], stocks[4], stocks[5], stocks[6], stocks[7], stocks[8]]
rinse_stocks = [stocks[9], stocks[10], stocks[11]]

## Check sample alignment

In [22]:
jubilee.pickup_tool(mix_syringe)
for well in stocks:
    jubilee.move_to(x = well.x, y = well.y)
    jubilee.move_to(z = well.top_+7)
    ans = input(f'Is the tip centered over well?')
    if ans == 'y':
        continue
    else:
        continue

Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 
Is the tip centered over well? 


In [17]:
deck.safe_z

45.0

In [29]:
jubilee.pickup_tool(teos_syringe)
for i, row in sample_table.iterrows():
    loc = samples[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

Is the tip centered over sample A1? 
Is the tip centered over sample A2? 
Is the tip centered over sample A3? 
Is the tip centered over sample A4? 
Is the tip centered over sample A5? 
Is the tip centered over sample A6? 
Is the tip centered over sample A7? 
Is the tip centered over sample A8? 
Is the tip centered over sample B1? 
Is the tip centered over sample B3? 
Is the tip centered over sample B4? 
Is the tip centered over sample B5? 
Is the tip centered over sample B6? 
Is the tip centered over sample B8? 
Is the tip centered over sample C1? 
Is the tip centered over sample C2? 
Is the tip centered over sample C3? 
Is the tip centered over sample C4? 
Is the tip centered over sample C5? 
Is the tip centered over sample C6? 


In [26]:
jubilee.park_tool()

# Experiment


In [54]:
location_lookup = {row['uuid']:samples[row['well']] for i, row in sample_table.iterrows()}

In [59]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,ba69b584-1afd-4436-b67b-41028e68893b,65.677050,68.946224,15.705124,1549.671602,0.577773,10.398547,0.012978,718.0,982.0,A1,C7
1,1,f25a70f2-6079-4953-8b41-51b60e129fe1,198.521614,129.936523,85.466756,1286.075107,0.191145,31.431624,0.038094,1375.0,325.0,A2,C8
2,2,f0a886af-f326-4c5f-ba18-ac69dd3177b9,17.873642,12.064687,128.255937,1541.805734,2.123038,2.829906,0.003505,0.0,1700.0,A3,D1
3,3,1468528a-c926-4cdf-8982-dc31a2cc397b,224.271870,84.116124,252.692166,1138.919840,0.169198,35.508622,0.042028,1412.0,288.0,A4,D2
4,4,1ac99397-b138-4836-a940-50a40a1ae83c,17.000000,37.210726,94.110614,1551.678661,2.232143,2.691584,0.003345,0.0,1700.0,A5,D3
5,5,38190afc-ea48-4c3b-8c2e-edd9cab5abf9,141.610206,117.375094,87.916288,1353.098413,0.267964,22.420927,0.027351,1244.0,456.0,A6,D4
6,6,f0f8a098-6c89-409b-8fe3-ffd4b4f106a8,17.000000,40.720692,170.658786,1471.620522,2.232143,2.691584,0.003305,0.0,1700.0,A7,D5
7,7,cbb3d083-994b-4677-b507-3cdc98c3861b,238.499852,166.129657,234.973235,1060.397256,0.159105,37.761317,0.044464,1430.0,270.0,A8,D6
8,8,1c6baba0-ebdd-4e85-982c-5f83fe191312,76.662951,155.652010,74.715308,1392.969730,0.494977,12.137928,0.014891,859.0,841.0,B1,D7
9,9,bdbbf131-7db4-4768-846e-2de75e623ef2,252.952528,99.754920,142.946974,1204.345578,0.150014,40.049587,0.047974,1445.0,255.0,B3,D8


In [55]:
dilution_table = sample_table.copy()

In [56]:
dilution_table['well'] = dilution_table['dilute_well']

In [58]:
dilution_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,ba69b584-1afd-4436-b67b-41028e68893b,65.677050,68.946224,15.705124,1549.671602,0.577773,10.398547,0.012978,718.0,982.0,C7,C7
1,1,f25a70f2-6079-4953-8b41-51b60e129fe1,198.521614,129.936523,85.466756,1286.075107,0.191145,31.431624,0.038094,1375.0,325.0,C8,C8
2,2,f0a886af-f326-4c5f-ba18-ac69dd3177b9,17.873642,12.064687,128.255937,1541.805734,2.123038,2.829906,0.003505,0.0,1700.0,D1,D1
3,3,1468528a-c926-4cdf-8982-dc31a2cc397b,224.271870,84.116124,252.692166,1138.919840,0.169198,35.508622,0.042028,1412.0,288.0,D2,D2
4,4,1ac99397-b138-4836-a940-50a40a1ae83c,17.000000,37.210726,94.110614,1551.678661,2.232143,2.691584,0.003345,0.0,1700.0,D3,D3
5,5,38190afc-ea48-4c3b-8c2e-edd9cab5abf9,141.610206,117.375094,87.916288,1353.098413,0.267964,22.420927,0.027351,1244.0,456.0,D4,D4
6,6,f0f8a098-6c89-409b-8fe3-ffd4b4f106a8,17.000000,40.720692,170.658786,1471.620522,2.232143,2.691584,0.003305,0.0,1700.0,D5,D5
7,7,cbb3d083-994b-4677-b507-3cdc98c3861b,238.499852,166.129657,234.973235,1060.397256,0.159105,37.761317,0.044464,1430.0,270.0,D6,D6
8,8,1c6baba0-ebdd-4e85-982c-5f83fe191312,76.662951,155.652010,74.715308,1392.969730,0.494977,12.137928,0.014891,859.0,841.0,D7,D7
9,9,bdbbf131-7db4-4768-846e-2de75e623ef2,252.952528,99.754920,142.946974,1204.345578,0.150014,40.049587,0.047974,1445.0,255.0,D8,D8


In [60]:
location_lookup_dilute = {}

for i, row in dilution_table.iterrows():
    uuid_val = row['uuid']
    well = row['well']
    if well is not np.nan:
        well_loc = samples[well]
    else:
        well_loc = None

    location_lookup_dilute[uuid_val] = well_loc

## add ethanol

In [63]:
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table, location_lookup, 'ethanol_volume', ethanol_stocks, stocks_usable_volume=15000, dwell_time = 10)

In [64]:
#dispense remaining volume from etOH mix syringe
jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

In [65]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)

## Dispense water


In [66]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table, location_lookup, 'water_volume', water_stocks, dwell_time = 7)

## dispense Ammonia

In [67]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table, location_lookup, 'ammonia_volume', ammonia_stocks)

## mix precursors

In [68]:
stober.first_mix(jubilee, mix_syringe, 2000, location_lookup, rinse_stocks, 5)

## Add TEOS and mix

In [69]:
teos_start_time = time.time()

In [70]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table, location_lookup, 'teos_volume', teos_stocks, mix_after=(2000, 5, rinse_stocks))

### Dilution

In [71]:
hrs5 = 60*60*5
while (time.time() - teos_start_time) < hrs5:
    time.sleep(60)

# dispese ethanol into dilution vials
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, dilution_table, location_lookup_dilute, 'ethanol_dilute_vol', ethanol_stocks[4:], stocks_usable_volume=15000, dwell_time = 10)

jubilee.pickup_tool(mix_syringe)
rinse_vol = 2000
for i, row in dilution_table.iterrows():
    uuid_val = row['uuid']
    sample_well = location_lookup[uuid_val]
    dilute_well = location_lookup_dilute[uuid_val]
    if dilute_well is None:
        continue
    else:
        vol = row['sample_dilute_vol']
        
        mix_syringe.aspirate(vol, sample_well.bottom(+5))
        time.sleep(10)
        mix_syringe.dispense(vol, dilute_well.bottom(+5))
        time.sleep(10)
        logger.info(f'Transferred {vol} uL from {sample_well} to {dilute_well}')
    
        for stock in rinse_stocks:
            mix_syringe.mix(rinse_vol, 5, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 500)
    
        logger.info(f'Washed mix syringe in wash solutions {rinse_stocks}')

    

AttributeError: 'NoneType' object has no attribute 'bottom'

In [80]:
jubilee.park_tool()

In [100]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1)

In [101]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1)

In [102]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1)

In [79]:
location_lookup

{'ba69b584-1afd-4436-b67b-41028e68893b': Well A1 form septavialrev1_44_holder_2000ul on slot 2,
 'f25a70f2-6079-4953-8b41-51b60e129fe1': Well A2 form septavialrev1_44_holder_2000ul on slot 2,
 'f0a886af-f326-4c5f-ba18-ac69dd3177b9': Well A3 form septavialrev1_44_holder_2000ul on slot 2,
 '1468528a-c926-4cdf-8982-dc31a2cc397b': Well A4 form septavialrev1_44_holder_2000ul on slot 2,
 '1ac99397-b138-4836-a940-50a40a1ae83c': Well A5 form septavialrev1_44_holder_2000ul on slot 2,
 '38190afc-ea48-4c3b-8c2e-edd9cab5abf9': Well A6 form septavialrev1_44_holder_2000ul on slot 2,
 'f0f8a098-6c89-409b-8fe3-ffd4b4f106a8': Well A7 form septavialrev1_44_holder_2000ul on slot 2,
 'cbb3d083-994b-4677-b507-3cdc98c3861b': Well A8 form septavialrev1_44_holder_2000ul on slot 2,
 '1c6baba0-ebdd-4e85-982c-5f83fe191312': Well B1 form septavialrev1_44_holder_2000ul on slot 2,
 'bdbbf131-7db4-4768-846e-2de75e623ef2': Well B3 form septavialrev1_44_holder_2000ul on slot 2,
 '81c188d3-242b-4913-b293-cf022d523bf1':

In [74]:
sample_well

Well C6 form septavialrev1_44_holder_2000ul on slot 2

In [75]:
dilute_well

In [76]:
row

Unnamed: 0                                                    19
uuid                        f9c5e673-ef4c-4e1e-832d-307ab7bd2d99
teos_volume                                            90.779998
ammonia_volume                                             13.94
water_volume                                          120.869994
ethanol_volume                                       1474.410008
dilution_volume_fraction                                0.418004
silica_mass_conc                                       14.373058
silica_mass_fraction                                    0.017684
ethanol_dilute_vol                                         989.0
sample_dilute_vol                                          711.0
well                                                         NaN
dilute_well                                                  NaN
Name: 19, dtype: object

In [81]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,ba69b584-1afd-4436-b67b-41028e68893b,65.677050,68.946224,15.705124,1549.671602,0.577773,10.398547,0.012978,718.0,982.0,A1,C7
1,1,f25a70f2-6079-4953-8b41-51b60e129fe1,198.521614,129.936523,85.466756,1286.075107,0.191145,31.431624,0.038094,1375.0,325.0,A2,C8
2,2,f0a886af-f326-4c5f-ba18-ac69dd3177b9,17.873642,12.064687,128.255937,1541.805734,2.123038,2.829906,0.003505,0.0,1700.0,A3,D1
3,3,1468528a-c926-4cdf-8982-dc31a2cc397b,224.271870,84.116124,252.692166,1138.919840,0.169198,35.508622,0.042028,1412.0,288.0,A4,D2
4,4,1ac99397-b138-4836-a940-50a40a1ae83c,17.000000,37.210726,94.110614,1551.678661,2.232143,2.691584,0.003345,0.0,1700.0,A5,D3
5,5,38190afc-ea48-4c3b-8c2e-edd9cab5abf9,141.610206,117.375094,87.916288,1353.098413,0.267964,22.420927,0.027351,1244.0,456.0,A6,D4
6,6,f0f8a098-6c89-409b-8fe3-ffd4b4f106a8,17.000000,40.720692,170.658786,1471.620522,2.232143,2.691584,0.003305,0.0,1700.0,A7,D5
7,7,cbb3d083-994b-4677-b507-3cdc98c3861b,238.499852,166.129657,234.973235,1060.397256,0.159105,37.761317,0.044464,1430.0,270.0,A8,D6
8,8,1c6baba0-ebdd-4e85-982c-5f83fe191312,76.662951,155.652010,74.715308,1392.969730,0.494977,12.137928,0.014891,859.0,841.0,B1,D7
9,9,bdbbf131-7db4-4768-846e-2de75e623ef2,252.952528,99.754920,142.946974,1204.345578,0.150014,40.049587,0.047974,1445.0,255.0,B3,D8


In [82]:
## TEOS calibration test

In [103]:
teos_vol = 50

old_wells = samples[0:4]
new_wells = samples['A6', 'A7', 'A8', 'B1', 'B3']

TypeError: list indices must be integers or slices, not tuple

In [84]:
old_wells

[Well A1 form septavialrev1_44_holder_2000ul on slot 2,
 Well A2 form septavialrev1_44_holder_2000ul on slot 2,
 Well A3 form septavialrev1_44_holder_2000ul on slot 2,
 Well A4 form septavialrev1_44_holder_2000ul on slot 2]

In [85]:
aspirate_volume = teos_syringe.capacity - teos_syringe.remaining_volume - 1

In [107]:
jubilee.pickup_tool(teos_syringe)

In [87]:
teos_syringe.aspirate(aspirate_volume, teos_stocks[0].bottom(+5), s = 10)

In [88]:
for well in old_wells:
    teos_syringe.dispense(teos_vol, well)

In [109]:
jubilee.park_tool()

In [104]:
new_wells = ['A6', 'A7', 'A8', 'B1', 'B3']

In [108]:
for ind in new_wells:
    well = samples[ind]
    teos_syringe.dispense(teos_vol, well)

In [106]:
well

Well B3 form septavialrev1_44_holder_2000ul on slot 2